In [2]:
import pandas as pd

In [3]:
bulk_datos="/home/franciscojavier/Escritorio/TFG/scDEAL/data/ALL_expression.csv"
Bulk= pd.read_csv(bulk_datos, index_col=0)

In [4]:
print(f"El archivo tiene {Bulk.shape[0]} filas y {Bulk.shape[1]} columnas.")


El archivo tiene 1280 filas y 15962 columnas.


In [5]:
print(f"Número de genes: {Bulk.shape[1]}")
print(f"Número de muestras: {Bulk.shape[0]}")
print("Primeros 5 genes:", Bulk.columns[:5].tolist())

Número de genes: 15962
Número de muestras: 1280
Primeros 5 genes: ['ST3GAL2', 'NOTCH2', 'GRPEL1', 'SHC2', 'MAGEA8']


# CALCULAMOS VARIANZA

In [6]:
varianzas = Bulk.var(axis=0)  # varianza por columna (gen)
print(varianzas)

ST3GAL2    0.892848
NOTCH2     0.813878
GRPEL1     2.395445
SHC2       3.767519
MAGEA8     2.661780
             ...   
RNF13      4.454409
LAMC1      2.325935
PITX3      2.773513
SNX24      1.557141
IL17RE     1.291969
Length: 15962, dtype: float64


In [7]:
top_20_porcentaje = int(len(varianzas) * 0.2) #Filtramos el top 20
genes_top = varianzas.sort_values(ascending=False).head(top_20_porcentaje).index
df_top_genes = Bulk[genes_top]

In [8]:
print(f"Genes seleccionados: {df_top_genes.shape[1]} de {Bulk.shape[1]} originales")


Genes seleccionados: 3192 de 15962 originales


# MAPEAMOS A RATON

In [10]:
#pip install mygene
import mygene
# Lista de genes humanos que quieres mapear 
genes_humanos = list(df_top_genes.columns)  # por ejemplo si tu DataFrame ya está filtrado

In [11]:
#Inicializamos
mg = mygene.MyGeneInfo()
# Buscar ortólogos en ratón
resultados = mg.querymany(genes_humanos, scopes='symbol', fields='ensembl.gene', species='mouse')


Input sequence provided is already in string format. No operation performed
Input sequence provided is already in string format. No operation performed
5 input query terms found dup hits:	[('EPPK1', 2), ('ST6GALNAC2', 2), ('MAGEF1', 2), ('NNT', 2), ('LCN15', 2)]
305 input query terms found no hit:	['RPS4Y1', 'S100P', 'C19ORF33', 'CLEC2B', 'HNRNPA1L2', 'HLA-DPA1', 'HLA-DRA', 'AKR1C1', 'AKR1C3', 'A


In [12]:
mapeo = {}
for r in resultados: #MApeamos solo aquellos q tiene 1 solo homologo en raton , los demas los descartamos que son los de la celda de arriba
    if 'notfound' not in r and 'ensembl' in r:
        # Puede haber uno o varios resultados (lista o dict)
        if isinstance(r['ensembl'], list):
            mapeo[r['query']] = r['ensembl'][0]['gene']
        else:
            mapeo[r['query']] = r['ensembl']['gene']

In [13]:
print(f"Genes mapeados: {len(mapeo)} de {len(genes_humanos)}")

Genes mapeados: 2883 de 3192


In [14]:
for humano, raton in list(mapeo.items())[:10]:
    print(f"{humano}  →  {raton}")

KRT19  →  ENSMUSG00000020911
VIM  →  ENSMUSG00000026728
EEF1A1  →  ENSMUSG00000037742
RPL18A  →  ENSMUSG00000045128
TACSTD2  →  ENSMUSG00000051397
RPS18  →  ENSMUSG00000008668
BASP1  →  ENSMUSG00000045763
RPS19  →  ENSMUSG00000040952
SRGN  →  ENSMUSG00000020077
UCHL1  →  ENSMUSG00000029223


In [21]:
genes_raton_bulk = list(mapeo.values())
len(genes_raton_bulk)


2883

In [22]:
mapeo_inverso = {v: k for k, v in mapeo.items()}  # ENSMUSG → gen humano


In [23]:
genes_humanos_ordenados = [mapeo_inverso[ensm] for ensm in genes_raton_bulk if ensm in mapeo_inverso]

In [24]:
df_bulk_humano_ordenado = Bulk[genes_humanos_ordenados]


In [25]:
df_bulk_humano_ordenado.head()

,KRT19,VIM,EEF1A1,RPL18A,TACSTD2,RPS18,BASP1,RPS19,SRGN,UCHL1,...,IL15,GALNT7,IGFL3,ANGPT1,RRP1B,IFNGR1,TCEAL5,DUSP11,GNA11,MFSD6
DATA.683665,3.405941,12.085272,4.922896,3.062681,3.496972,4.855941,5.697826,4.038713,11.725963,5.737451,...,4.384003,7.540249,3.394853,3.002283,8.001636,7.831898,3.023507,7.876978,4.013298,7.270762
DATA.683667,4.226230,11.661481,6.533475,3.113093,3.027207,4.680126,5.011485,4.045782,3.211200,9.158466,...,3.449861,7.249157,3.379538,3.001898,8.308363,7.443246,3.019110,7.700416,8.309896,4.358078
DATA.684052,4.032691,12.671484,5.551361,2.892388,2.832211,3.884602,11.453684,3.944376,2.873959,10.270623,...,3.000656,8.260060,3.307964,3.387063,7.957953,6.905754,3.405001,7.464873,8.508063,4.455841
DATA.684057,3.091820,10.667924,5.277746,2.932124,2.932878,4.880253,9.359950,3.950865,2.909001,10.074418,...,3.783932,6.288173,3.390363,2.994032,8.219939,8.016595,4.280432,7.801002,7.849750,3.848845
DATA.684059,3.492342,12.870585,5.819835,2.982949,2.923305,3.888214,10.924775,3.717019,2.676016,11.074848,...,2.849301,7.708803,3.518197,3.736578,7.958470,8.002485,3.412617,6.972215,8.140381,4.290012


In [26]:
df_bulk_humano_ordenado.to_csv("bulk_2883_humano.csv")

# RESULTADOS

In [27]:
import scanpy as sc

adata = sc.read("/home/franciscojavier/Escritorio/TFG/scDEAL/save/adata/GSE110894_integrate_data_GSE110894_drug_I.BET.762_bottle_512_edim_256,128_pdim_128,64_model_DAEIntento3_dropout_0.1_gene_F_lr_0.5_mod_new_sam_upsampling.h5ad")  # reemplaza <para> por el sufijo real del modelo


Only considering the two last: ['.5_mod_new_sam_upsampling', '.h5ad'].
Only considering the two last: ['.5_mod_new_sam_upsampling', '.h5ad'].


In [28]:
y_true = adata.obs['sensitive']             # Etiqueta verdadera
y_pred = adata.obs['sens_label'].astype(int)  # Etiqueta predicha por el modelo

In [29]:
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

print("Accuracy:", accuracy_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred, average='weighted'))
print("Precision:", precision_score(y_true, y_pred, average='weighted'))
print("Recall:", recall_score(y_true, y_pred, average='weighted'))

# Reporte completo
print(classification_report(y_true, y_pred))


Accuracy: 0.9436222692036645
F1 Score: 0.9435148150391073
Precision: 0.9450743402741256
Recall: 0.9436222692036645
              precision    recall  f1-score   support

           0       0.97      0.91      0.94       685
           1       0.92      0.97      0.95       734

    accuracy                           0.94      1419
   macro avg       0.95      0.94      0.94      1419
weighted avg       0.95      0.94      0.94      1419

